How many rented units were affordable to households at different incomes in 1980, 1990, 2000, 2010 and 2023?

In [14]:
import pandas as pd
import numpy as np

IPUMS v3 includes household income, built year 1 and 2, bedrooms, rent, gross rent for 1980, 1990, 2000, 2005 (ACS), 2010 (ACS), 2014 (ACS), 2023 (ACS) 

In [11]:
# load ipums data
df = pd.read_csv('source/ipums_v3.csv', low_memory=False)
df.head(3)

,YEAR,SAMPLE,SERIAL,CBSERIAL,HHWT,CLUSTER,CPI99,CITY,STRATA,GQ,OWNERSHP,OWNERSHPD,RENT,RENTGRS,HHINCOME,BUILTYR,BUILTYR2,BEDROOMS
0,1980,198002,550665,NaN,100,1980005506652,2.295,1190,21,1,2,22,165,165,16005,7.0,NaN,2
1,1980,198002,550666,NaN,100,1980005506662,2.295,1190,21,1,2,22,212,235,18005,5.0,NaN,2
2,1980,198002,550667,NaN,100,1980005506672,2.295,1190,35,1,2,22,325,415,12535,6.0,NaN,5


In [12]:
# cleanup

# convert year to string
df['YEAR'] = df['YEAR'].astype(str)

# remove duplicate household serials
df.drop_duplicates(subset=['SERIAL'], keep='first', inplace=True)

In [15]:
# filter out NA income
df['adj_hhincome'] = np.where(df['HHINCOME']!=9999999,df['HHINCOME']/12, np.nan)

In [16]:
# adjust rents and incomes for inflation with these constants: https://usa.ipums.org/usa/cpi99.shtml
# adjust to 1999 base year
# for acs years, most recent year is the data dollar year

df['adj_rent'] = (df['RENTGRS'] * df['CPI99']) # adjust to 1999
df['adj_hhincome'] = (df['HHINCOME'] * df['CPI99']) # adjust to 1999

# adjust from 1999 base year to 2024 with BLS CPI-U
# https://www.minneapolisfed.org/about-us/monetary-policy/inflation-calculator/consumer-price-index-1913-
df['adj_rent'] = df['adj_rent'] * (314.4/166.6) # adjust to 2024
df['adj_hhincome'] = df['adj_hhincome'] * (314.4/166.6) # adjust to 2024

In [17]:
# check total households looks right?
df.groupby('YEAR')['HHWT'].sum()

YEAR
1980    1140300
1990    1007025
2000    1124221
2005     868164
2010    1055521
2014     880277
2023    1170677
Name: HHWT, dtype: int64

In [21]:
# create pivot the number of households at each rent
pivot = pd.pivot_table(df[df['adj_rent'] != 0],
              index='adj_rent',
              columns='YEAR',
              values='HHWT',
              aggfunc='sum')

pivot.to_csv('processed/rent_pivot_by_year.csv')

pivot = pivot.reset_index()
pivot

YEAR,adj_rent,1980,1990,2000,2005,2010,2014,2023
0,5.314228,NaN,NaN,NaN,NaN,NaN,209.0,NaN
1,18.871549,NaN,NaN,108.0,NaN,NaN,NaN,NaN
2,20.758703,NaN,NaN,123.0,NaN,NaN,NaN,NaN
3,26.420168,NaN,NaN,124.0,NaN,NaN,NaN,NaN
4,28.835726,NaN,NaN,NaN,NaN,150.0,NaN,NaN
...,...,...,...,...,...,...,...,...
4080,5644.472643,NaN,NaN,NaN,NaN,NaN,NaN,99.0
4081,5883.960144,NaN,NaN,NaN,NaN,NaN,NaN,83.0
4082,5966.542041,NaN,NaN,NaN,NaN,NaN,NaN,79.0
4083,5987.187515,NaN,NaN,NaN,NaN,NaN,NaN,95.0


In [22]:
# load 2023-adjusted AMIs for each year
# source: NHGIS
# https://docs.google.com/spreadsheets/d/1C3ToVnNv3JRd01gKtjIfcY_IeDyKyOS1pvqflSLynAU/edit?usp=sharing

ami = {'1980': 64197, # decennial
       '1990': 64623, # decennial
       '2000': 70612, # decennial
       '2010': 62539, # acs 1-year,
       '2014': 62622, #acs 1-year
       '2023': 74474} # acs 1-year

In [23]:
# create a df that has the % of affordable units at each 10% AMI interval for all years

ami_pcts = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2]

dfdict = {'year': [],
         'ami': [],
         'ami_pct': [],
         'ami_pct_value': [],
         'ami_pct_monthly': [],
         'ami_pct_aff_threshold': [],
         'aff_units': [],
         'pct_aff_units': []
         }

for year in ami.keys():
    for pct in ami_pcts:
        dfdict['year'].append(year)
        dfdict['ami'].append(ami[year])
        dfdict['ami_pct'].append(pct)
        
        ami_pct_value = ami[year] * pct # %ami
        dfdict['ami_pct_value'].append(ami_pct_value)
        
        ami_pct_monthly = ami_pct_value / 12 # %ami per month
        dfdict['ami_pct_monthly'].append(ami_pct_monthly)
        
        ami_pct_aff_threshold = ami_pct_monthly * 0.3 # 30% of monthly income
        dfdict['ami_pct_aff_threshold'].append(ami_pct_aff_threshold)

        aff_units = pivot.loc[pivot['adj_rent'] < ami_pct_aff_threshold, year].sum() # calc num of rentals less than threshold
        dfdict['aff_units'].append(aff_units)

        pct_aff_units = aff_units / pivot[year].sum()
        dfdict['pct_aff_units'].append(pct_aff_units)

In [24]:
output = pd.DataFrame(dfdict)

In [31]:
output.head(20)

,year,ami,ami_pct,ami_pct_value,ami_pct_monthly,ami_pct_aff_threshold,aff_units,pct_aff_units
0,1980,64197,0.1,6419.7,534.975,160.4925,3000.0,0.004727
1,1980,64197,0.2,12839.4,1069.950,320.9850,25700.0,0.040498
2,1980,64197,0.3,19259.1,1604.925,481.4775,49600.0,0.078159
3,1980,64197,0.4,25678.8,2139.900,641.9700,89100.0,0.140403
4,1980,64197,0.5,32098.5,2674.875,802.4625,174000.0,0.274188
5,1980,64197,0.6,38518.2,3209.850,962.9550,297000.0,0.468011
6,1980,64197,0.7,44937.9,3744.825,1123.4475,416000.0,0.655531
7,1980,64197,0.8,51357.6,4279.800,1283.9400,496700.0,0.782698
8,1980,64197,0.9,57777.3,4814.775,1444.4325,543300.0,0.856130
9,1980,64197,1.0,64197.0,5349.750,1604.9250,577500.0,0.910022


In [30]:
output.tail(20)

,year,ami,ami_pct,ami_pct_value,ami_pct_monthly,ami_pct_aff_threshold,aff_units,pct_aff_units
100,2023,74474,0.1,7447.4,620.616667,186.185,2257.0,0.003794
101,2023,74474,0.2,14894.8,1241.233333,372.370,22105.0,0.037157
102,2023,74474,0.3,22342.2,1861.850000,558.555,38117.0,0.064072
103,2023,74474,0.4,29789.6,2482.466667,744.740,56534.0,0.095030
104,2023,74474,0.5,37237.0,3103.083333,930.925,108007.0,0.181552
105,2023,74474,0.6,44684.4,3723.700000,1117.110,178120.0,0.299407
106,2023,74474,0.7,52131.8,4344.316667,1303.295,241156.0,0.405366
107,2023,74474,0.8,59579.2,4964.933333,1489.480,311554.0,0.523699
108,2023,74474,0.9,67026.6,5585.550000,1675.665,362846.0,0.609917
109,2023,74474,1.0,74474.0,6206.166667,1861.850,409813.0,0.688866


A WBEZ analysis finds that a household making the city’s median income in 1980 could afford more than percent of apartments in the city. That means they would spend no more than roughly a third of their monthly income on rent and utilities. By 2023, just under 70 percent of apartments were affordable. 

In [32]:
# export as csv
output.to_csv('output/aff_units_by_ami_threshold.csv')